In [1]:
from __future__ import division

In [2]:
import numpy as np
import math

In [110]:
class Quaternion():
    def __init__(self, r=0, i=0, j=0, k=0):
        self.m = np.array([
            [r, -i, -j, k], 
            [i, r, -k, -j], 
            [j, k, r, i], 
            [-k, j, -i, r]]).astype(float)

    @staticmethod
    def as_array(matrix):
        return np.append(matrix[:3, 0], -matrix[3, 0])
        
    @staticmethod
    def from_array(array):
        return Quaternion(*list(array))

    @staticmethod
    def from_matrix(matrix):
        q = Quaternion()
        q.m = matrix
        return q

    def coords(self):
        return [self.s(), self.i(), self.j(), self.k()]
    
    def s(self):
        return self.m[0, 0]

    def i(self):
        return self.m[1, 0]
        
    def j(self):
        return self.m[2, 0]
        
    def k(self):
        return -self.m[3, 0]
        
    def norm(self):
        return math.sqrt((self * self.conj()).s())
        
    def __mul__(self, other):
        if isinstance(other, Quaternion):
            return Quaternion.from_matrix(np.matmul(self.m, other.m))
        elif isinstance(other, int) or isinstance(other, float):
            return Quaternion.from_matrix(self.m * other)
    
    def __truediv__(self, other):
        if isinstance(other, Quaternion):
            return self * other.conj() / (other * other.conj()).s()
        elif isinstance(other, int) or isinstance(other, float):
            return Quaternion.from_matrix(self.m / other)
    
    def __add__(self, other):
        return Quaternion.from_matrix(self.m + other.m)
    
    def __sub__(self, other):
        return Quaternion.from_matrix(self.m - other.m)
    
    def __neg__(self):
        return Quaternion.from_matrix(-self.m)
    
    def conj(self):
        return Quaternion.from_matrix(np.transpose(self.m))

    def __str__(self):
        return str(Quaternion.as_array(self.m))
    
    def __repr__(self):
        return str(Quaternion.as_array(self.m))

    def __eq__(self, other):
        return np.array_equal(self.m, other.m)

class Q(Quaternion):
    pass


One = Q(1, 0, 0, 0)
i = Q(0, 1, 0, 0)
j = Q(0, 0, 1, 0)
k = Q(0, 0, 0, 1)

In [106]:
q = Q(j=3)
q, q.s(), q.j(), q.k(), -q

([ 0.  0.  3. -0.], 0.0, 3.0, -0.0, [-0. -0. -3.  0.])

In [5]:
print(i, i*i)
print(j, j*j)
print(k, k*k)
print(i * j, j * k, i * k)

([ 0.  1.  0. -0.], [-1.  0.  0. -0.])
([ 0.  0.  1. -0.], [-1.  0.  0. -0.])
([0. 0. 0. 1.], [-1.  0.  0. -0.])
([0. 0. 0. 1.], [ 0.  1.  0. -0.], [ 0.  0. -1. -0.])


In [6]:
i.conj(), j*j.conj()

([ 0. -1.  0. -0.], [ 1.  0.  0. -0.])

In [7]:
k == Q(k=1)

True

In [8]:
A = Q(1, 2, 3, 4)
B = Q(1, -1, 2, 3)
(A*B).conj() == B.conj()*A.conj()

True

In [9]:
k*2, One/k

([0. 0. 0. 2.], [ 0.  0.  0. -1.])

In [10]:
inv = One / Q(1, 1)
Q(1, 1), inv, inv * Q(1, 1), inv.norm(), Q(1, 1).norm()

([ 1.  1.  0. -0.],
 [ 0.5 -0.5  0.  -0. ],
 [ 1.  0.  0. -0.],
 0.7071067811865476,
 1.4142135623730951)

In [11]:
A.s(), A.i(), A.j(), A.k()

(1.0, 2.0, 3.0, 4.0)

In [89]:
def eapp(q, precision=20):
    qq = Q(1)
    res = Q(1)
    for i in range(1, precision):
        qq = qq * q
        res = res + qq * 1/math.factorial(i)
    return res

In [90]:
eapp(j*math.pi), qexp_approx(Q(2)), math.exp(2)

([-1.00000000e+00  0.00000000e+00 -5.28918724e-10 -0.00000000e+00],
 [ 7.3890561  0.         0.        -0.       ],
 7.38905609893065)

---
Demonstrating: $e^{q_1 + q_2} \neq e^{q_1} \cdot e^{q_2}$

In [91]:
eapp(i+j), eapp(i) * eapp(j)

([ 0.15594369  0.698456    0.698456   -0.        ],
 [0.29192658 0.45464871 0.45464871 0.70807342])

In [93]:
eapp(i+k), eapp(j+k)

([0.15594369 0.698456   0.         0.698456  ],
 [0.15594369 0.         0.698456   0.698456  ])

---
Demonstrating: $a = r \cdot (cos\theta + \mathbf{u}sin\theta)$

In [58]:
q = Q(1,3,1,-1)
a0=q.s()
a1=q.i()
a2=q.j()
a3=q.k()
na = math.sqrt(a1*a1 + a2*a2 + a3*a3)
r = math.sqrt(a0*a0 + a1*a1 + a2*a2 + a3*a3)
print(r)
cos = a0/r
print(cos)
sin=na/r  
print(sin)
a = Q(cos, a1/na*sin, a2/na*sin, a3/na*sin) * r
a

3.46410161514
0.2886751345948129
0.957427107756


[ 1.  3.  1. -1.]

In [101]:
i*j

[0. 0. 0. 1.]

In [102]:
i*k

[ 0.  0. -1. -0.]

In [98]:
i*i

[-1.  0.  0. -0.]

In [99]:
i*Q(1,1)

[-1.  1.  0. -0.]

In [107]:
p=[1, 1, 2]

In [108]:
Q(0, *p)

[0. 1. 1. 2.]

---
#### Rotations in 3D
Rotating any point ```p=[x,y,z]``` by ```deg``` degrees (units of 360°) around ```axis=[ax, ay, az]```

We know that a rotation around axis $\mathbf{u}$ of a quaternion $q \in \mathcal{Vec}(\mathbb{H}) $ is given by $ w\cdot q \cdot w^{-1}$

With $ w = (cos\frac{\theta}{2} + sin\frac{\theta}{2}\cdot \mathbf{u})$, $|w|=1 $

and $ w^\dagger = w^{-1} = (cos\frac{\theta}{2} - sin\frac{\theta}{2}\cdot \mathbf{u})$

Using $q^{\prime} = (cos\frac{\theta}{2} + sin\frac{\theta}{2}\cdot \mathbf{u})\cdot q \cdot(cos\frac{\theta}{2} - sin\frac{\theta}{2}\cdot \mathbf{u})$

In [121]:
def rotate(p, axis, deg):
    u = Q(0, *axis)
    u = u / u.norm()
    q = Q(0, *p)
    cos=math.cos(deg*math.pi/360)
    sin=math.sin(deg*math.pi/360)
    w=Q(cos) +u*sin
    q_ = w * q * w.conj()
    return q_.coords()[1:]

In [132]:
print rotate([1, 0, 0], [0,0,1], 90)
print rotate([0, 1, 0], [0,0,1], 90)

print rotate([0, 1, 0], [1,0,0], 90)
print rotate([0, 0, 1], [1,0,0], 90)

print rotate([0, 0, 1], [0,1,0], 90)
print rotate([1, 0, 0], [0,1,0], 90)

print rotate([1,-1,1], [1,1,1], 90)

[2.220446049250313e-16, 1.0, -0.0]
[-1.0, 2.220446049250313e-16, -0.0]
[0.0, 2.220446049250313e-16, 1.0]
[0.0, -1.0, 2.220446049250313e-16]
[1.0, 0.0, 2.220446049250313e-16]
[2.220446049250313e-16, 0.0, -1.0]
[1.4880338717125852, 0.33333333333333326, -0.8213672050459183]
